# Jamie's Phase 1 Project Work!
### hooray 

In [13]:
# imports
import pandas as pd
import numpy as mp
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')

In [16]:
imdb_names = pd.read_csv('zippedData/imdb.name.basics.csv.gz')

In [17]:
imdb_titles = pd.read_csv('zippedData/imdb.title.akas.csv.gz')

In [ ]:
imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')

In [ ]:
imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')

In [ ]:
imdb_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')